In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import depthcharge as dc
import numpy as np
from depthcharge.encoders import PeakEncoder
from depthcharge.data import SpectrumDataset, StreamingSpectrumDataset

This method works, but not ideal.

In [21]:
mzml = './orbitrap_lumos/08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML' #'..//08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML'

dataset = SpectrumDataset(mzml, batch_size=54852)
encoder = PeakEncoder(100)

08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML: 100%|██████████| 54852/54852 [00:25<00:00, 2118.90 spectra/s]


This is the ideal way to read in spectra for the problem we are trying to solve, but I'll see if I can get it to work.

In [29]:
import os

orbi_lumos_directory = 'orbitrap_lumos'
orbi_lumos_mzmls = [os.path.join(orbi_lumos_directory, file) 
             for file in os.listdir(orbi_lumos_directory) 
             if os.path.isfile(os.path.join(orbi_lumos_directory, file))]
q_exactive_hfx_directory = 'Q_exactive_hfx'
q_exactive_hfx_mzmls = [os.path.join(q_exactive_hfx_directory, file) 
             for file in os.listdir(q_exactive_hfx_directory) 
             if os.path.isfile(os.path.join(q_exactive_hfx_directory, file))]
# Assign labels for each list
ms_labeler = {}

# Add files from list1 with label 0
for filename in orbi_lumos_mzmls:
    ms_labeler[filename] = orbi_lumos_directory

# Add files from list2 with label 1
for filename in q_exactive_hfx_mzmls:
    ms_labeler[filename] = q_exactive_hfx_directory

all_mzmls = orbi_lumos_mzmls + q_exactive_hfx_mzmls

#orbi_lumos_mzmls = './orbitrap_lumos/08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML'
processing_fn = [
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.set_mz_range
    dc.data.preprocessing.set_mz_range(min_mz=0),
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.filter_intensity
    dc.data.preprocessing.filter_intensity(min_intensity = 0.1),  # Change this value to allow number of peaks per spectrum
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.scale_intensity
    dc.data.preprocessing.scale_intensity(scaling=None),  # Might want to change later
    dc.data.preprocessing.scale_to_unit_norm,  # Not sure what this is yet
]

all_data = list()
all_labels = list()
for mzml_file in all_mzmls:
    # Preprocess the mzML file
    df = dc.data.spectra_to_df(
        mzml_file,
        progress=True,
        preprocessing_fn=processing_fn
    )
    
    # Encode peaks
    encoder = PeakEncoder(100)
    dataset = StreamingSpectrumDataset(df, batch_size=54852)
    embeddings = None

    for spectrum in dataset:
        mz_values = spectrum["mz_array"]
        intensities = spectrum["intensity_array"]
        stack = torch.stack((mz_values, intensities), dim=2)
        embedded_batch = encoder.forward(stack)
        embeddings = embedded_batch.detach()
        
        # Append the data and labels
        all_data.append(embeddings)
        all_labels.extend([ms_labeler[mzml_file]] * embeddings.size(0))  # Repeat the label for all spectra
'''  
df = dc.data.spectra_to_df(
    mzml,
    progress=True,
    preprocessing_fn=processing_fn
)
encoder = PeakEncoder(100)
dataset = SpectrumDataset(df, batch_size=54852)
'''

BD20180604_PTRC_DP1-Microscale_SS_Plex3_Proteome_F20.mzML: 100%|██████████| 49246/49246 [00:19<00:00, 2581.68 spectra/s]
08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML: 100%|██████████| 54852/54852 [00:25<00:00, 2189.76 spectra/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9b in position 367: invalid start byte

Not run right now

In [25]:
embeddings = None
for spectrum in dataset:
    mz_values = spectrum["mz_array"]
    intensities = spectrum["intensity_array"]
    stack = torch.stack((mz_values, intensities), dim= 2)
    embedded_batch = encoder.forward(stack)
    embeddings = embedded_batch.detach()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
'''
# 1. Data Preparation
# Assuming 'labels' is a numpy array of shape (39793,) with 0 or 1 for each sample
# Example:
np.random.seed(42)
num_samples = 39793
labels = np.random.randint(0, 2, num_samples)
num_epochs = 3

# Load your data (replace this with your actual data loading)
data = embeddings

# Convert labels to a PyTorch tensor
labels = torch.from_numpy(labels).long()

# Create TensorDataset and DataLoader
dataset = TensorDataset(data, labels)
batch_size = 1  # Per DC, should only be 1, but this takes longer. Originally 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
'''

num_epochs = 3
# Stack all embeddings into a single tensor
all_data_tensor = torch.cat(all_data, dim=0)

# Convert labels to a PyTorch tensor
all_labels_tensor = torch.tensor(all_labels, dtype=torch.long)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(all_data_tensor, all_labels_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 2. Model Definition (CNN approach - recommended)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1)  # Input channels = 1
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 50 * 27, 2) #nn.Linear(64 * 50 * 25, 2)  # Adjust this depending on the input

    def forward(self, x):
        # Ensure input has correct dimensions
        if x.dim() == 3:  # Handle input shape (N, H, W)
            x = x.unsqueeze(1)  # Add channel dimension
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

# For Rob
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# For Nvidia GPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Classifier().to(device)

# 3. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop with reshaping for CNN
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Reshape inputs for CNN
        if inputs.dim() == 3:
            inputs = inputs.unsqueeze(1)  # (N, H, W) -> (N, C, H, W)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# During evaluation, move inputs and labels to MPS (or CUDA)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        # Move data to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)

        # Perform forward pass
        outputs = model(inputs)

        # Compute predictions and accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Epoch [1/3], Loss: 0.7182
Epoch [2/3], Loss: 0.6950
Epoch [3/3], Loss: 0.6766
Accuracy: 49.91%
